# MinPrj_Lyrics

date : 5 october 2021
author : bae hueng myoung
prj goals : 
    1)
    2)


In [10]:
import os, re, glob
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/*'
txt_list = glob.glob(txt_file_path)
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])



데이터 크기: 187088
Examples:
 ['Can we forget about the things I said when I was drunk...', "I didn't mean to call you that", "I can't remember what was said"]


In [2]:
print(txt_list)

['/aiffel/aiffel/lyricist/data/blink-182.txt', '/aiffel/aiffel/lyricist/data/al-green.txt', '/aiffel/aiffel/lyricist/data/johnny-cash.txt', '/aiffel/aiffel/lyricist/data/r-kelly.txt', '/aiffel/aiffel/lyricist/data/nirvana.txt', '/aiffel/aiffel/lyricist/data/dickinson.txt', '/aiffel/aiffel/lyricist/data/michael-jackson.txt', '/aiffel/aiffel/lyricist/data/paul-simon.txt', '/aiffel/aiffel/lyricist/data/prince.txt', '/aiffel/aiffel/lyricist/data/nicki-minaj.txt', '/aiffel/aiffel/lyricist/data/adele.txt', '/aiffel/aiffel/lyricist/data/Lil_Wayne.txt', '/aiffel/aiffel/lyricist/data/lorde.txt', '/aiffel/aiffel/lyricist/data/dj-khaled.txt', '/aiffel/aiffel/lyricist/data/bob-marley.txt', '/aiffel/aiffel/lyricist/data/alicia-keys.txt', '/aiffel/aiffel/lyricist/data/janisjoplin.txt', '/aiffel/aiffel/lyricist/data/lady-gaga.txt', '/aiffel/aiffel/lyricist/data/dolly-parton.txt', '/aiffel/aiffel/lyricist/data/britney-spears.txt', '/aiffel/aiffel/lyricist/data/lin-manuel-miranda.txt', '/aiffel/aiffel/

# preprocessing
    1) 불필요한 문자 제거
    2) length < 13 : 토큰화 갯수가 15개 이하를 만족하기 위해선 각 문장의 공백은 13개 이하가 되어야 함
    3) generating token
    4) word frequency caculation / 입려된 문장들의 단어들이 얼마나의 빈도를 갖는지 계산

In [11]:
# 1) 불필요한 문자 제거
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [12]:
# 2) length < 13 : 토큰화 갯수가 15개 이하를 만족하기 위해선 각 문장의 공백은 13개 이하가 되어야 함
def blank_num_calc(sentence):
    cnt = 0
    for ele in sentence:
        if ele == ' ':
            cnt += 1
    return cnt
    
    
lenmodcorp = []
for sen in raw_corpus:
    if blank_num_calc(sen) < 13:
        lenmodcorp.append(sen)

print(len(raw_corpus))
print(len(lenmodcorp))
print(lenmodcorp[:10])


187088
174144
['Can we forget about the things I said when I was drunk...', "I didn't mean to call you that", "I can't remember what was said", 'Or what you threw at me Please tell me', 'Please tell me why', 'My car is in the front yard', 'And I am sleeping with my cloths on', 'I came in throught the window... Last night', 'And your... Gone', "Gone It's no suprise to me I am my own worst enemy"]


In [13]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence_ele in lenmodcorp:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence_ele) == 0: continue
    if sentence_ele[-1] == ":": continue
    
    
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence_ele)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]


['<start> can we forget about the things i said when i was drunk . . . <end>',
 '<start> i didn t mean to call you that <end>',
 '<start> i can t remember what was said <end>',
 '<start> or what you threw at me please tell me <end>',
 '<start> please tell me why <end>',
 '<start> my car is in the front yard <end>',
 '<start> and i am sleeping with my cloths on <end>',
 '<start> i came in throught the window . . . last night <end>',
 '<start> and your . . . gone <end>',
 '<start> gone it s no suprise to me i am my own worst enemy <end>']

In [14]:
# if token num > 15, remove sentence!
corpus_new = []
for sen in corpus:
    if blank_num_calc(sen) < 15:
        corpus_new.append(sen)
        
print(len(corpus_new))

155793


In [15]:
# 3) generating token
# making num_words = 12000
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus_new)
tensor = tensor[:,:15]
print(tensor.shape)
print(tokenizer)

(155793, 15)


In [16]:
word_list = tokenizer.index_word

for idx in word_list:
    print(idx, ":", word_list[idx])
    if idx >= 20: break



1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : that
16 : t
17 : s
18 : on
19 : your
20 : of


In [8]:
# 4) word frequency caculation
def countword(word,cor):
    splitted = cor.split(' ')
    cnt = 0
    for spi in splitted:
        if word == spi:
            cnt += 1
    return cnt


for word in word_list.values():
    cnt_r = 0
    for cor in corpus:
        cnt_r += countword(word,cor)
    print(word,cnt_r)


<unk> 0
<start> 162813
<end> 162813
, 53789
i 50661
the 41496
you 38345
and 24463
a 22205
to 21802
it 18449
me 17212
my 16256
in 14029
t 13264
s 12974
that 12554
on 10488
. 9697
your 9576
of 9511
we 9146
m 9102
like 8301
all 8101
is 7699
be 7205
for 6928
up 6837
with 6631
so 6560
can 6233
but 6219
know 6216
just 6211
don 6163
love 6017
no 5874
what 5723
got 5714
oh 5568
they 5530
this 5484
? 5455
get 5346
she 5138
when 5065
do 4888
yeah 4710
now 4374
baby 4314
re 4294
if 4257
go 4134
he 4115
out 4036
was 3962
! 3845
down 3755
one 3635
ll 3509
there 3271
see 3185
let 3177
want 3129
cause 3099
come 3093
not 3066
her 3042
say 2990
at 2962
make 2943
from 2798
time 2793
have 2742
are 2720
back 2711
how 2609
never 2555
girl 2501
im 2490
take 2454
as 2439
man 2331
will 2314
right 2286
way 2256
wanna 2223
u 2179
ain 2157
ve 2108
here 2071
need 2070
then 2034
who 2004
tell 1998
some 1956
more 1925
gonna 1925
where 1877
his 1821
too 1819
been 1789
could 1771
feel 1770
them 1759
life 1739
or 1714

KeyboardInterrupt: 

In [17]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    


from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input,test_size=0.2,random_state=7)
print(enc_train.shape)
print(src_input[0])
print(tgt_input[0])

(124634, 14)
[  2   4 380  16 251  10 151   7  15   3   0   0   0   0]
[  4 380  16 251  10 151   7  15   3   0   0   0   0   0]


# making data obj

In [18]:
BUFFER_SIZE = len(enc_train)
BATCH_SIZE = 256
steps_per_epoch = len(enc_train) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

# Making NLP model

In [19]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size_256 = 256
embedding_size_512 = 512
embedding_size_1024 = 1024

hidden_size_1024 = 1024

model_256 = TextGenerator(tokenizer.num_words + 1, embedding_size_256 , hidden_size_1024)
model_512 = TextGenerator(tokenizer.num_words + 1, embedding_size_512 , hidden_size_1024)
model_1024 = TextGenerator(tokenizer.num_words + 1, embedding_size_1024 , hidden_size_1024)

In [20]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model_256(src_sample)
model_512(src_sample)
model_1024(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-3.4075015e-04,  6.3131127e-05,  3.1429663e-04, ...,
         -3.3353126e-04,  3.7346571e-04, -1.7741224e-04],
        [-1.5780328e-05,  5.9107144e-04,  4.9672404e-04, ...,
         -5.5604958e-04,  2.8807513e-04, -3.0980541e-04],
        [ 5.8825733e-04,  5.4587051e-04,  8.5185032e-04, ...,
          3.3580576e-04,  8.3170235e-05,  7.2784816e-05],
        ...,
        [-7.5425072e-05, -3.8450971e-04,  7.2748640e-05, ...,
          1.1259964e-03, -8.8564947e-04,  1.7225716e-03],
        [ 4.1206953e-05, -1.0404268e-03, -1.7760563e-04, ...,
          9.1674284e-04, -8.9251599e-04,  1.1176531e-03],
        [-8.2374696e-05, -1.5279116e-03, -3.3882435e-04, ...,
          5.3805608e-04, -1.2416464e-03,  2.3276308e-04]],

       [[-3.4075015e-04,  6.3131127e-05,  3.1429663e-04, ...,
         -3.3353126e-04,  3.7346571e-04, -1.7741224e-04],
        [-9.7924785e-04, -2.0659184e-04,  5.5647304e-04, ...,
         -6.4967055e-04, 

# Learning

In [21]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model_256.compile(loss=loss, optimizer=optimizer)
model_256.fit(dataset, epochs=10)

Epoch 1/10
486/486 [==============================] - 163s 329ms/step - loss: 4.0523
Epoch 2/10
486/486 [==============================] - 160s 329ms/step - loss: 3.0661
Epoch 3/10
486/486 [==============================] - 159s 327ms/step - loss: 2.8847
Epoch 4/10
486/486 [==============================] - 159s 326ms/step - loss: 2.7488
Epoch 5/10
486/486 [==============================] - 162s 333ms/step - loss: 2.6363
Epoch 6/10
486/486 [==============================] - 159s 326ms/step - loss: 2.5385
Epoch 7/10
486/486 [==============================] - 159s 327ms/step - loss: 2.4552
Epoch 8/10
486/486 [==============================] - 159s 326ms/step - loss: 2.3720
Epoch 9/10
486/486 [==============================] - 159s 328ms/step - loss: 2.2977
Epoch 10/10
486/486 [==============================] - 159s 327ms/step - loss: 2.2213


In [22]:
model_512.compile(loss=loss, optimizer=optimizer)
model_512.fit(dataset, epochs=10)

Epoch 1/10
486/486 [==============================] - 171s 347ms/step - loss: 3.9029
Epoch 2/10
486/486 [==============================] - 169s 348ms/step - loss: 3.0658
Epoch 3/10
486/486 [==============================] - 170s 349ms/step - loss: 2.8941
Epoch 4/10
486/486 [==============================] - 170s 349ms/step - loss: 2.7732
Epoch 5/10
486/486 [==============================] - 170s 349ms/step - loss: 2.6491
Epoch 6/10
486/486 [==============================] - 170s 349ms/step - loss: 2.5280
Epoch 7/10
486/486 [==============================] - 175s 359ms/step - loss: 2.4210
Epoch 8/10
486/486 [==============================] - 170s 349ms/step - loss: 2.3081
Epoch 9/10
486/486 [==============================] - 170s 349ms/step - loss: 2.1961
Epoch 10/10
486/486 [==============================] - 170s 349ms/step - loss: 2.0894


In [23]:
model_1024.compile(loss=loss, optimizer=optimizer)
model_1024.fit(dataset, epochs=10)

Epoch 1/10
486/486 [==============================] - 202s 409ms/step - loss: 3.9199
Epoch 2/10
486/486 [==============================] - 199s 409ms/step - loss: 3.1663
Epoch 3/10
486/486 [==============================] - 199s 410ms/step - loss: 3.0049
Epoch 4/10
486/486 [==============================] - 199s 410ms/step - loss: 2.9197
Epoch 5/10
486/486 [==============================] - 200s 410ms/step - loss: 2.8563
Epoch 6/10
486/486 [==============================] - 199s 410ms/step - loss: 2.8114
Epoch 7/10
486/486 [==============================] - 200s 410ms/step - loss: 2.7601
Epoch 8/10
486/486 [==============================] - 200s 410ms/step - loss: 2.7216
Epoch 9/10
486/486 [==============================] - 199s 410ms/step - loss: 2.6764
Epoch 10/10
486/486 [==============================] - 200s 411ms/step - loss: 2.6361


# generate text!

In [24]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [30]:
generate_text(model_256, tokenizer, init_sentence="<start> she")

'<start> she s got me runnin round and round <end> '

In [31]:
generate_text(model_512, tokenizer, init_sentence="<start> she")

'<start> she s got me runnin round and round <end> '

In [32]:
generate_text(model_1024, tokenizer, init_sentence="<start> she")

'<start> she s a <unk> <end> '

# overview
	기계는 사람과 친구처럼 대화할 수 있을까? 매일 터미널 앞에서 혹은 코랩에 마주할 때면 기계의 생각을 읽고 싶을 때가 많다. “내가 명령어를 만들어 주면 너는 왜 항상 불평과 불만을 주는것이냐?, 너는 내가 하는 말들을 왜 못알아 먹는 것이냐?” 기계는 3살짜리 어린아이를 달래주는 것과 비슷하다. 기계를 설득하기 위해서 전적으로 나의 행동들이 모두 잘못된 것이다. 들여쓰기 하나 못한 것도 기계의 입장에서는 중대한 대역죄로 취급된다. 나의 행동들을 교정하면서 기계를 설득하다 보면 녹초가 되곤 한다. 하지만 오늘의 NLP모델은 미래에 컴퓨터와의 대화에서 이러한 스트레스에 자유를 줄지도 모른다는 생각이 든다. 사람의 문장들을 학습시켜 적절한 문장을 구사하는 아주 기초적인 실습이지만 기게와의 대화는 이제까지 사람의 명령에 묵묵히 수행했던 답답하고 투박한 상자에서 그들의 생각을 물어볼 수 있는 기술로 기대된다. 
	이 실습에서의 목적은 embedding과 lstm의 rnn의 알고리즘을 이용하여 단어들을 학습시키고 하나의 단어를 입력하였을 때 적절한 문장을 출력하는 것이다. 이 실습에서 embedding의 기능과 하이퍼파라마미터를 변경하면서 학습 성능을 평가할 수 있는 loss의 값의 추이를 알아볼 것이며 2.2 이하로 낮추도록 설계해 볼 것이다.

# Design

	aiffel에서 제공한 각종 팝송의 가사들을 데이터를 입력시키고 각 가사들의 단어들을 적절한 벡터와 토큰화시키는 전처리(preprocessing)작업을 수행한다. 그리고 단어들의 유사성을 계산하고 표현하기 위해 one-hot encoding대신 embdding 기술을 이용하며 TFmaster에서 배운 LTSM으로 이전의 처리된 정보들을 참조하여 단어들 간의 유사성을 계산한다. 

# Analysis
	전처리 과정에서 lms에서 제공한 전처리 함수를 그대로 이용하였다. 불필요한 특수문자들을 제거하고 일련의 NLPmodel을 형성하였다. 이 때 모델의 loss(epoch=10)는 2.42로 2.2이하의 목표에 미치지 못했다. 하지만 토큰의 갯수를 15개로 제한하였을 때 모델의 loss는 2.2213으로 성능 개선이 이루어졌다. 이는 cs231에서 배운 backpropagation으로 gradient를 계산하지 않아도 입력값의 변화는 2.42-2.2213 = (about) 0.2의 차이를 보임을 확인할 수 있다. 이번에는 embdding size를 변화해보기로 하였다. 256, 512, 1024로 2배씩 늘려 model_256, model_512, model_1024로 나누어 성능을 평가해보았다. 

emodding size/time/loss : 	256/160s/2.422
						512/170s/2.0894
						1024/200s/2.6361

이러한 embedding size의 변화는 loss의 변화를 나태냄이 뚜렷하였다. embdding size가 512일 때 가장 성능이 좋았고 1024일 때 가장 최악의 성능을 보였다. 이는 임베딩 사이즈가 클수록 좋은 것이 아닌 특정 최적의 지점이 있음을 내포하고 있다. 이를 구체적으로 알아보기 위해선 embedding size의 범위를 충분히 확보하고 적절한 간격으로 엠베딩 사이즈를 확장하여 성능의 변화를 확인하여야 한다. 하지만 이는 매우 비용이 큰 작업이다. 에폭 10을 모두 소화하는데 무려 30분이나 걸린다. 따라서 임베딩 사이즈는 무턱대고 일일이 값들을 설정하여 최적의 사이즈를 찾는 것은 효율적이지 못하다. 

	그리고 각 모델에 “she”를 입력해보았다. model_256과 model_512는 “<start> she s got me runnin round and round <end>”라는 다소 어색하지만 그럴듯한 문장을 만들어내었다. 반면 model_1024는 “<start> she s a <unk> <end>”라는 문장을 만들어냈으며 문장을 모두 완성해내지 못했다. 이 모델은 loss가 2.6361로 가장 성능이 저조하였고 이정도의 loss는 문장하나를 만들어내는 적합한 모델을 만들지 못한다는 것을 알 수 있다. 


# Limits
	히든 사이즈의 변화에 따라 어떠한 loss의 변화를 보이는지 확인해보아야 했다.
	최적의 embedding size를 찾는 방법에 대해 강구했어야 했다.
	어색한 문장에 대해 원인을 파악하고 해결했어야 했다. (she s got me runnin round and round는 실제 사람의 대화에서 표현하지 않는다. )
	
# In my opinion
	(이 글은 학습시키는데 너무 무료한 나머지 개인적인 생각을 적은 것들입니다. 무료함 속에서 자유로운 생각과 상상의 나래를 펼칠 수 있는 좋은 경험인것 같습니다.)


	embedding 기술은 머신러닝에서 해당 카테고리값들을 숫자와 벡터로 변환하여 유사성과 표현효율성을 높이는데 매우 탁월한 면모를 보인다는 것은 매우 흥미롭다. LMS에서 배운 one hot encoding과 구글에서 얻은 정보들을 종합해보면 oen hot encdoing은 0과 1의 표현으로 카테고리가 늘어나면 필요한 벡터공간은 면적배로 늘어나는 단점을 지닌다. 해당 카테고리를 의미하는 1은 겨우 1개인데 나머지는 0으로 매핑되는 것은 데이터 공간의 비효율성이 한눈에 보인다. 더군다나 각 카테고리들의 유사성을 one hot encoding으로 표현하기 역부족이다. 기계는 유사성을 각 카테고리의 밸류의 거리에 대해서 인지할 수 있는데 one hot encoding은 각 카테고리의 거리들은 동일하다. (예를 들면 3차원 공간에서 (1,0,0)의 점과 (0,1,0)의 점 (0,0,1)의 세 점의 거리는 모두 동일하다)
	이러한 한계에 대해서 embedding 기술은 자연어의 단어들 간의 유사성을 표현하는데 매우 탁월한 기술임을 코딩을 통해 확인할 수 있었다. 모델을 만들고 하나의 배치사이즈의 데이터를 입력해보면 단어들의 encoding된 값들이 표현되며 이 값들을 토대로 단어들의 유사성을 직관적으로 확인할 수 없지만 결과 값들의 차이로 단어들 간의 유사성을 표현할 수 있음을 알 수 있다. 

	하지만 나는 사람이 대화를 할 때의 반응속도에 대해서 생각해보았다. 나는 LMS를 하면서 팝송을 즐겨 듣는다. 그리고 오늘 들었던 팝송들 중 유난히 빠른 곡이 하나 있었다. jake miller의 rumors는 다소 빠른 속도의 비트와 함께 음악이 진행된다. 나는 영어가 서툴러 이러한 단어들을 모두 이해할 수 없지만 반면 기초적 교육 과정을 거친 미국인들은 아무렇지 않듯 이 노래들을 흥겨듣거나 부르기까지 한다. 영어에 익숙한 미국인들이 한국인보다 더 빠르게 영어 단어들을 이해하고 구사할 수 있는 이유는 학습된 경험의 양이 많기 때문이다. 경험에서 어떤 상황에서 어떤 단어들과 표현을 구사할 것인가에 대해서 패턴들을 축적해왔고 끊임없이 더 나은 표현의 데이터베이스를 구축해 왔을 것이다. 그리고 특정 상황에 닥칠 때 반사적으로, 무의식적으로 가장 적합한 단어와 표현을 만들어낸다. 
	내가 했던 이 NLP model은 얼마나 빠르게 반응하여 문장들을 자연스럽게 구사할 수 있을까? 오늘 해본 embedding과 lstm은 단어들의 유사성을 모두 계산하고 특정 단어를 입력하면 가장 유사성이 높은 단어들을 출력하여 하나의 문장을 완성한다. 하지만 적절한 문장을 만드는데는 문장들의 데이터셋을 집어넣어 빈도와 유사성만으로는 사람과 같이 복잡하고 어려운 문장들을 구사하기 힘들것이다. 그리고 학습된 데이터의 수가 많은수록 계산해야 할 유사성은 더욱 더 복잡해지며 최종적으로 출력할 때의 계산량은 아마도 엄청난 시간과 비용을 지불해야 할 지 모른다.
	반면 사람은 적합한 단어들을 구사할 때 단어들의 유사성의 값들을 계산하고 값들을 일일이 확인하지 않는다. 우리가 문장을 구사할 때 계산기를 두드려가며 문장의 적합도를 계산하는 대신 감각에 의존한다. 아주 짧은 찰나의 시간동안 구사할 문장에 대해서 느낌으로 적합한지 그렇지 않은지 시뮬레이션 하며 문장 구성의 모든 과정은 감각 의존적이다. 그렇다면 기계도 복잡한 수식에 의존하는 것보다 생물학적 감각으로 문장 구사과정을 설계할 수 없을까? 두뇌의 뉴런과 시냅스의 화학반응과 같이 감각반응이 장착된다면 감각으로 불필요한 단어들을 한번에 문장 과정에 고려하지 않고 몇가지만 추려 빠르게 적합한 문장을 구사할 수 있지 않을까? 만약 이것이 가능하다면 매우 많은 데이터에 대해서 시간과 비용을 아낄 수 있을지 모른다. 

	